In [ ]:
# Este notebook possui instruções importantes, mas não recomendo executá-lo localmente porque o Kernel do Jupyter Notebook pode travar por falta de memória.
# Recomendo executar este notebook no Google Colab: https://colab.research.google.com/drive/17aipZlLd-t1WNXKh7jlcDQY8Kg8v-0zB

# NOTE. As etapas para preparar o dataset estão disponíveis em: https://colab.research.google.com/drive/1N8SINqJgnPaD1oddg49O1xyy6BkoI7Op

# YOLOv8 - Treinamento de modelo customizado

Lembrando que o dataset que criamos em aula já se encontra devidamente anotado e convertido para o formato usado pelo YOLO. Isso foi explicado detalhadamente na aula sobre criação do dataset, onde além de fazer a preparação é feito o download de imagens do Open Images Dataset direto pelo Colab.

Caso deseje treinar outros objetos:
* Pode procurar pelo [Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html) ou [Kaggle](https://www.kaggle.com) por exemplo, se há um dataset para a classe que deseja detectar.
* Ou pode criar você mesmo o dataset.
Além do LabelImg (que mostramos no curso como é possível fazer a anotação) segue outras ferramentas para fazer a anotação direto pelo navegador e assim não precisar baixar nenhum programa:
  * [MakeSense.ai](https://www.makesense.ai), [CVAT](https://www.cvat.ai), ou ainda a plataforma do [Roboflow](https://app.roboflow.com/login) (inclusive eles oferecem uma integração mais direta com YOLOv8).

Obs: se o seu dataset estiver em outro formato de anotação que não seja do YOLO nem o padrão usado pelo OID, então você pode adaptar o nosso script de conversão de anotações, ou ainda dar uma olhada nas ferramentas que o Roboflow oferece para converter o formato das anotações: https://roboflow.com/formats/yolov8-pytorch-txt

In [3]:
import os
import sys

helpers_path = os.path.abspath("..")

sys.path.append(helpers_path)

In [4]:
!nvidia-smi

Sun Dec 28 08:58:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.91                 Driver Version: 581.32         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   53C    P8              4W /   85W |    4582MiB /   6144MiB |     26%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Preparação do dataset

A estrutura necessária é a seguinte

* /dataset
  * /train (conjunto de treino)
  * /val (conjunto de teste)

In [ ]:
from helpers.settings import SHARED_PATH

dataset_dir = f'{SHARED_PATH}/dataset/data'

## Instalação das ferramentas do YOLOv8

In [6]:
# !pip install ultralytics

In [7]:
from ultralytics import YOLO
import os
import cv2
import matplotlib.pyplot as plt

## Configurações do arquivo YAML

Agora, precisamos preencher esse arquivo com os parâmetros necessários

* path: [caminho do diretório que contém o dataset]
* train: [caminho do conjunto de treinamento, relativo ao path]
* val: [caminho do conjunto de validação, relativo ao path]
* test: [não é necessário pois faremos depois um teste manual]

**Número de Classes**
* nc: [coloca o numero de classes que deseja treinar]

**Nomes/Labels - substitua pelos nomes das classes**
* names: [nome de cada classe, dentro de '' e separado por vírgula]

Para escrever esses valores no arquivo, podemos usar o comando %%writefile

In [9]:
content = f"""path: '{dataset_dir}'
train: 'obj/'
val: 'valid/'
test: # opcional

nc: 4
names: ['Person', 'Car', 'Dog', 'Cat']
"""
with open('configs_modelo.yaml', 'w') as f:
    f.write(content)

## Treinamento do modelo

In [10]:
from helpers.settings import NOTEBOOKS_PATH

diretorio_raiz = NOTEBOOKS_PATH
arquivo_config = os.path.join(diretorio_raiz, 'configs_modelo.yaml')

In [11]:
arquivo_config

'/home/rapha/Learn/opencv-yolo-darknet-course-backup/src/notebooks/configs_modelo.yaml'

In [ ]:
# !yolo task=detect mode=train data={arquivo_config} epochs=10

> **Parâmetros da função de treinamento:**

* task: o que queremos com o treinamento. Como estamos trabalhando com detecção de objetos, deixe o valor = detect. Outras opções aceitas: segment, e classify. É opcional passarmos se queremos detecção, pois por padrão ele já considera como sendo detecção a não ser que especifique outra forma.
* mode: pode ser train, val, ou predict. Como estamos fazendo pela forma usando python e queremos o treinamento vamos usar a função train(), portanto esse parâmetro se torna desnecessário .
* **model**: o modelo pré-treinado que queremos usar como "partida". Pode ser o YOLOv8 Nano (YOLOv8n), YOLOv8 Small (YOLOv8s), etc.
* **imgsz**: O tamanho da imagem, que a rede realiza o processamento (obs: você não precisa redimensionar a imagem para esse tamanho antes, o algoritmo cuida dessa parte antes de passar a imagem de entrada para a rede). A resolução padrão é 640x640 pixels, portanto o valor padrão é 640. Quando maior o tamanho mais precisa é a detecção, principalmente para objetos com detalhes pequenos, porém é mais demorado o treinamento e detecção.  
* **data**: caminho para o arquivo YAML. Esse é o arquivo que criamos acima, que contém o caminho para o conjunto de treinamento e validaçao, além disso deve conter os nomes das classes que queremos treinar.
* **epochs**: Numero de epocas que desejamos treinar.
* **batch**: O tamanho do batch (lote) para o carregador de dados. Você pode aumentá-lo ou diminuí-lo de acordo com a disponibilidade de memória de sua GPU, por exemplo caso venha a encontrar problema de memória. O valor padrão é 16.
* **name**: Nome do diretório de resultados para o runs/detect. (opcional)

Em nossos testes vamos escolher o Nano, ou até mesmo o Small, pois queremos que seja um treinamento relativamente mais rápido

In [12]:
model = YOLO('yolov8n.yaml')

In [ ]:
resultados = model.train(data=arquivo_config, epochs=10, imgsz=640, name='yolov8n_modelo', model='yolov8n.pt')

In [ ]:
dir_resultado = '/content/runs/detect/yolov8s_modelo'

### Avaliação (Verificando o mAP do modelo)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!yolo task=detect mode=val model={dir_resultado}/weights/best.pt name=yolov8s_modelo_eval data=configs_modelo.yaml

### Exibindo os gráficos

In [ ]:
def mostrar(img):
  fig = plt.gcf()
  fig.set_size_inches(16, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
#resultados_grafico

In [ ]:
resultados_grafico = cv2.imread(os.path.join(dir_resultado, 'results.png'))
mostrar(resultados_grafico)

In [ ]:
dir_resultado_val = 'runs/detect/yolov8s_modelo_eval'

In [ ]:
imgs = ['F1_curve.png', 'PR_curve.png', 'P_curve.png', 'R_curve.png']
plt.figure(figsize=(18,14))
for i, img in enumerate(imgs):
  #print(i, img)
  grafico = cv2.imread(os.path.join(dir_resultado_val, img))
  #print(grafico)
  grafico = cv2.cvtColor(grafico, cv2.COLOR_BGR2RGB)
  plt.subplot(2, 2, i + 1)
  plt.title(imgs[i])
  plt.imshow(grafico)
  plt.axis('off')
plt.show()

In [ ]:
matriz_confusao = cv2.imread(os.path.join(dir_resultado_val, 'confusion_matrix.png'))
mostrar(matriz_confusao)

## Testando o modelo treinado


In [ ]:
!mkdir imagens_teste

In [ ]:
!yolo task=detect mode=predict model={dir_resultado}/weights/best.pt source='/content/imagens_teste' save=true conf=0.05

In [ ]:
dir_predicoes = 'runs/detect/predict4/'
caminhos = [os.path.join(dir_predicoes, f) for f in os.listdir(dir_predicoes)]
#print(caminhos)
for caminho_imagem in caminhos:
  imagem = cv2.imread(caminho_imagem)
  mostrar(imagem)

## Continuar treinamento



In [ ]:
!yolo task=detect mode=train model={dir_resultado}/weights/last.pt data={arquivo_config} epochs=10

In [ ]:
!yolo task=detect mode=val model={dir_resultado}/weights/best.pt name=yolov8s_modelo_eval data=configs_modelo.yaml

In [ ]:
!yolo task=detect mode=predict model=runs/detect/train/weights/best.pt source='imagens_teste' save=true

In [ ]:
dir_predicoes = 'runs/detect/predict5/'
caminhos = [os.path.join(dir_predicoes, f) for f in os.listdir(dir_predicoes)]
#print(caminhos)
for caminho_imagem in caminhos:
  imagem = cv2.imread(caminho_imagem)
  mostrar(imagem)

## Enviar para o Google Drive

In [ ]:
!cp -R {dir_resultado} /content/gdrive/MyDrive/Cursos\ -\ recursos/YOLO/yolov8

## Exportar para outros formatos

In [ ]:
os.path.join(dir_resultado, 'weights', 'best.pt')

In [ ]:
model_treinado = YOLO(os.path.join(dir_resultado, 'weights', 'best.pt'))
model_treinado.export(format='onnx')

Confira a tabela com os formatos aceitos e qual o valor do parâmetro deve ser usado para salvar no formato específico

| Format                                                                     | `format=`          | Model                     |
|----------------------------------------------------------------------------|--------------------|---------------------------|
| [PyTorch](https://pytorch.org/)                                            | -                  | `yolov8n.pt`              |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)                    | `torchscript`      | `yolov8n.torchscript`     |
| [ONNX](https://onnx.ai/)                                                   | `onnx`             | `yolov8n.onnx`            |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)                     | `openvino`         | `yolov8n_openvino_model/` |
| [TensorRT](https://developer.nvidia.com/tensorrt)                          | `engine`           | `yolov8n.engine`          |
| [CoreML](https://github.com/apple/coremltools)                             | `coreml`           | `yolov8n.mlmodel`         |
| [TensorFlow SavedModel](https://www.tensorflow.org/guide/saved_model)      | `saved_model`      | `yolov8n_saved_model/`    |
| [TensorFlow GraphDef](https://www.tensorflow.org/api_docs/python/tf/Graph) | `pb`               | `yolov8n.pb`              |
| [TensorFlow Lite](https://www.tensorflow.org/lite)                         | `tflite`           | `yolov8n.tflite`          |
| [TensorFlow Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | `edgetpu`          | `yolov8n_edgetpu.tflite`  |
| [TensorFlow.js](https://www.tensorflow.org/js)                             | `tfjs`             | `yolov8n_web_model/`      |
| [PaddlePaddle](https://github.com/PaddlePaddle)                            | `paddle`           | `yolov8n_paddle_model/`   |



